In [0]:
# Sebastian Kuźmiński - sk149029
# Problem 1: Generowanie tekstu za pomocą łańcuchów Markowa (wiersz Szymborskiej)
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, sum as spark_sum
import random
import string
spark = SparkSession.builder.appName("MarkovChains").getOrCreate() # inicjalizacja sparka

# model Markowa przy użyciu PySpark DataFrame z argumentami tokens (słowa) i order (długość rzędu k=1 lub k=2)
def mk_model(tokens, order=1):
    ext_tokens = tokens + tokens[:order] # po ostatnim słowie może być pierwszy (cykliczność)
    # tworzenie par
    pairs = []
    for i in range(len(tokens)):
        ctx = tuple(ext_tokens[i:i+order]) # ctx = kontekst (słowo)
        nxt = ext_tokens[i+order] # nxt = następne słowo
        pairs.append(ctx + (nxt,)) # połączenie w parę
    
    # utworzenie kolumn z parami
    schema = [f"ctx_{i}" for i in range(order)] + ["nxt"]
    df = spark.createDataFrame(pairs, schema)
    ctx_cols = [f"ctx_{i}" for i in range(order)] # grupowanie i obliczanie prawdopodobieństw
    
    # liczenie wystąpień każdej pary (kontekst, nxt)
    counts_df = (
        df.groupBy(ctx_cols + ["nxt"])
        .agg(count("*").alias("cnt"))
    )
    # liczenie wszystkich wystąpień każdego kontekstu
    totals_df = (
        counts_df.groupBy(ctx_cols)
        .agg(spark_sum("cnt").alias("tot"))
    )
    # obliczenie prawdopodobieństw
    model_df = (
        counts_df.join(totals_df, ctx_cols)
        .withColumn("prob", col("cnt") / col("tot"))
    )
    return model_df

# generowanie sekwencji używając funkcji mk_model (model_df), start (słowo/słowa startowe), order (rząd k=1 lub k=2), seq_len (ilość wygenerowanych słów).
def gen_seq(model_df, start, order, seq_len=500):
    if order == 1: # dla k=1
        curr = start # słowo start
        result = [curr] # lista z wygenerowanymi słowami
    else:
        curr = start # dla k=2
        result = list(curr) # lista z wygenerowanymi słowami, konwertowanie krotki na listę, żeby móc dodawać elementy
    
    # generowanie kolejnych tokenów
    for _ in range(seq_len - order):
        # filtrowanie DataFrame po kontekście
        if order == 1: # dla k=1
            filter_cond = col("ctx_0") == curr # zwrócenie wierszy w kolumnie ctx_0, które mają słowo startowe
        else:
            filter_cond = None
            for i in range(order): # dla k=2, zwrócenie pary słów z kolumn ctx_0 i ctx_1
                cond = col(f"ctx_{i}") == curr[i]
                filter_cond = cond if filter_cond is None else filter_cond & cond
        
        rows = model_df.filter(filter_cond).collect() # pobranie możliwych następnych słów wraz z szansą
        
        if not rows: # jeżeli nie znaleziono żadnych możliwych przejść następuje przerwanie
            break
        
        # losowanie i łączenie następnego słowa według prawdopodobieństwa
        nexts = [r["nxt"] for r in rows]
        probs = [r["prob"] for r in rows]
        nxt = random.choices(nexts, weights=probs)[0]
        result.append(nxt) # połączenie z następnym słowem
        
        # aktualizacja kontekstu
        if order == 1: # dla k=1 wylosowane wcześniej słowo staje się słowem startowym
            curr = nxt
        else:
            curr = tuple(result[-order:]) # dla k=2 szukamy kolejnej pary słów
    return " ".join(result) # połączenie nowych słów

# funkcje statystyczne
def avg_len(text): # 1. średnia długość słowa (ilość znaków)
    tokens = text.split()
    if not tokens:
        return 0
    return sum(len(t) for t in tokens) / len(tokens)

def ttr(text): # 2. różnorodność leksykalna (udział słów występujących tylko raz)
    tokens = text.split()
    if not tokens:
        return 0
    return len(set(tokens)) / len(tokens) # set = usuwanie duplikatów

In [0]:
# wiersz "Dusza się miewa" Wisławy Szymborskiej
raw_text = """Duszę się miewa.
Nikt nie ma jej bez przerwy i na zawsze.
Dzień za dniem, rok za rokiem
może bez niej minąć.
Czasem tylko w zachwytach
i lękach dzieciństwa
zagnieżdża się na dłużej.
Czasem tylko w zdziwieniu,
że jesteśmy starzy.
Rzadko nam asystuje
podczas zajęć żmudnych,
jak przesuwanie mebli,
dźwiganie walizek,
czy przemierzanie drogi w ciasnych butach.
Przy wypełnianiu ankiet
i siekaniu mięsa
z reguły ma wychodne.
Na tysiąc naszych rozmów uczestniczy w jednej
a i to niekoniecznie, bo woli milczenie.
Kiedy ciało zaczyna nas boleć i boleć,
cichcem schodzi z dyżuru.
Jest wybredna: niechętnie widzi nas w tłumie,
mierzi ją nasza walka o byle przewagę
i terkot interesów.
Radość i smutek
to nie są dla niej dwa różne uczucia.
Tylko w ich połączeniu jest przy nas obecna.
Możemy na nią liczyć
kiedy niczego nie jesteśmy pewni,
a wszystkiego ciekawi.
Z przedmiotów materialnych
lubi zegary z wahadłem
i lustra, które pracują gorliwie,
nawet gdy nikt nie patrzy.
Nie mówi skąd przybywa
i kiedy znowu nam zniknie,
ale wyraźnie czeka na takie pytania.
Wygląda na to,
że tak jak ona nam,
również i my
jesteśmy jej na coś potrzebni."""

# usunięcie interpunkcji
trans = str.maketrans('', '', string.punctuation)
clean_text = raw_text.translate(trans)

# zamiana na małe litery i podział na tokeny (słowa)
tokens = clean_text.lower().split()

# tekst źródłowy (wykorzystywany do miar)
src_text = " ".join(tokens)

In [0]:
# MODEL K=1
order = 1
mk1 = mk_model(tokens, order=order)
gen1 = []

# generowanie sekwencji
for i in range(1): #  sekwencja
    start_tok = random.choice(tokens) # losowe słowo startowe
    seq = gen_seq(mk1, start_tok, order, seq_len=500)
    gen1.append(seq)
    
    print(f"Słowo start: '{start_tok}', długość {len(seq.split())} słów")
    print("Wylosowana sekwencja:")
    print(" ".join(seq.split()[:500]))

Słowo start: 'ona', długość 500 słów
Wylosowana sekwencja:
ona nam asystuje podczas zajęć żmudnych jak przesuwanie mebli dźwiganie walizek czy przemierzanie drogi w zachwytach i smutek to niekoniecznie bo woli milczenie kiedy ciało zaczyna nas boleć i kiedy niczego nie mówi skąd przybywa i my jesteśmy jej na to niekoniecznie bo woli milczenie kiedy niczego nie ma wychodne na to niekoniecznie bo woli milczenie kiedy niczego nie są dla niej dwa różne uczucia tylko w tłumie mierzi ją nasza walka o byle przewagę i kiedy znowu nam również i na takie pytania wygląda na zawsze dzień za dniem rok za dniem rok za rokiem może bez niej dwa różne uczucia tylko w tłumie mierzi ją nasza walka o byle przewagę i lękach dzieciństwa zagnieżdża się miewa nikt nie mówi skąd przybywa i lękach dzieciństwa zagnieżdża się miewa nikt nie ma wychodne na coś potrzebni duszę się miewa nikt nie patrzy nie ma wychodne na takie pytania wygląda na coś potrzebni duszę się na to że tak jak ona nam również i terkot inte

In [0]:
# MODEL K=2
order = 2
mk2 = mk_model(tokens, order=order)
gen2 = []

# generowanie sekwencji
for i in range(1): # jedna sekwencja
    idx = random.randint(0, len(tokens) - 2) # losowa para słów jako start
    start_pair = (tokens[idx], tokens[idx+1])
    seq = gen_seq(mk2, start_pair, order, seq_len=500)
    gen2.append(seq)
    
    print(f"Słowa start: {start_pair}, długość: {len(seq.split())} słów)")
    print("Wylosowana sekwencja:")
    print(" ".join(seq.split()[:500]))

Słowa start: ('niej', 'minąć'), długość: 500 słów)
Wylosowana sekwencja:
niej minąć czasem tylko w ich połączeniu jest przy nas obecna możemy na nią liczyć kiedy niczego nie jesteśmy pewni a wszystkiego ciekawi z przedmiotów materialnych lubi zegary z wahadłem i lustra które pracują gorliwie nawet gdy nikt nie patrzy nie mówi skąd przybywa i kiedy znowu nam zniknie ale wyraźnie czeka na takie pytania wygląda na to że tak jak ona nam również i my jesteśmy jej na coś potrzebni duszę się miewa nikt nie patrzy nie mówi skąd przybywa i kiedy znowu nam zniknie ale wyraźnie czeka na takie pytania wygląda na to że tak jak ona nam również i my jesteśmy jej na coś potrzebni duszę się miewa nikt nie patrzy nie mówi skąd przybywa i kiedy znowu nam zniknie ale wyraźnie czeka na takie pytania wygląda na to że tak jak ona nam również i my jesteśmy jej na coś potrzebni duszę się miewa nikt nie patrzy nie mówi skąd przybywa i kiedy znowu nam zniknie ale wyraźnie czeka na takie pytania wygląda na to że 

In [0]:
# miary statystyczne
# połączenie sekwencji dla każdego modelu
comb1 = " ".join(gen1)
comb2 = " ".join(gen2)
# tabela podsumowująca
print("\n" + "="*60)
print("TABELA PODSUMOWUJĄCA")
print("="*60)
print(f"{'Model':<15} {'Śr. długość słów':<20} {'Unikalność słów':<15}")
print("-"*60)
print(f"{'Źródło':<15} {avg_src:<20.2f} {ttr_src:<15.4f}")
print(f"{'K=1':<15} {avg_o1:<20.2f} {ttr_o1:<15.4f}")
print(f"{'Kr=2':<15} {avg_o2:<20.2f} {ttr_o2:<15.4f}")
print("="*60)

# interpretacja
print(f"  - Model K={1 if abs(avg_o1-avg_src) < abs(avg_o2-avg_src) else 2} jest bliższy źródłu (dla wylosowanych słów)")

print("\nType-Token Ratio (mierzy różnorodność: TTR = unikalne / wszystkie):")
print(f"  - Model K={1 if abs(ttr_o1-ttr_src) < abs(ttr_o2-ttr_src) else 2} jest bliższy TTR źródła (dla wylosowanych słów)")


TABELA PODSUMOWUJĄCA
Model           Śr. długość słów     Unikalność słów
------------------------------------------------------------
Źródło          4.85                 0.7181         
K=1             4.90                 0.2520         
Kr=2            4.64                 0.2600         
  - Model K=1 jest bliższy źródłu (dla wylosowanych słów)

Type-Token Ratio (Mierzy różnorodność: TTR = unikalne / wszystkie):
  - Model K=2 jest bliższy TTR źródła (dla wylosowanych słów)
